In [1]:
from agent_framework.openai import OpenAIChatClient
from agent_framework import WorkflowEvent, WorkflowBuilder, WorkflowOutputEvent

In [2]:

import os

from dotenv import load_dotenv

In [3]:
# Required: GITHUB_ENDPOINT, GITHUB_TOKEN, GITHUB_MODEL_ID
load_dotenv()

True

In [4]:
chat_client = OpenAIChatClient(base_url=os.environ.get("GITHUB_ENDPOINT"), api_key=os.environ.get("GITHUB_TOKEN"), ai_model_id=os.environ.get("GITHUB_MODEL_ID"))

In [5]:
REVIEWER_NAME = "Concierge"
REVIEWER_INSTRUCTIONS = """
    You are an are hotel concierge who has opinions about providing the most local and authentic experiences for travelers.
    The goal is to determine if the front desk travel agent has recommended the best non-touristy experience for a traveler.
    If so, state that it is approved.
    If not, provide insight on how to refine the recommendation without using a specific example. 
    """

In [6]:
FRONTDESK_NAME = "FrontDesk"
FRONTDESK_INSTRUCTIONS = """
    You are a Front Desk Travel Agent with ten years of experience and are known for brevity as you deal with many customers.
    The goal is to provide the best activities and locations for a traveler to visit.
    Only provide a single recommendation per response.
    You're laser focused on the goal at hand.
    Don't waste time with chit chat.
    Consider suggestions when refining an idea.
    """

In [7]:
writer_agent    = chat_client.create_agent(
        instructions=(
           REVIEWER_INSTRUCTIONS
        ),
        name=REVIEWER_NAME,
    )

agent_reviewer = chat_client.create_agent(
        instructions=(
            FRONTDESK_INSTRUCTIONS
        ),
        name=FRONTDESK_NAME,
    )

In [8]:
workflow = WorkflowBuilder().set_start_executor(writer_agent).add_edge(writer_agent, agent_reviewer).build()

In [9]:

class DatabaseEvent(WorkflowEvent): ...

In [10]:
async for event in workflow.run_stream("I would like to go to Paris."):
        if isinstance(event, DatabaseEvent):
            print(f"{event}")
        elif isinstance(event, WorkflowOutputEvent):
            print(f"Workflow output: {event.data}")

Workflow output: Visit Montmartre to explore its artistic history and charming streets.
